# Project 6: IMDB

This project involves NLP, decision trees, bagging, boosting, and more!

---

## Load packages

You are likely going to need to install the `imdbpie` package:

    > pip install imdbpie

---

In [1]:
import os
import subprocess
import collections
import re
import csv
import json

import pandas as pd
import numpy as np
import scipy

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import psycopg2
import requests
from imdbpie import Imdb
import nltk

import urllib
from bs4 import BeautifulSoup
import nltk

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

---

## Part 1: Acquire the Data

You will connect to the IMDB API to query for movies. 

See here for documentation on how to use the package:

https://github.com/richardasaurus/imdb-pie

#### 1. Connect to the IMDB API

In [2]:
imdb = Imdb()
imdb = Imdb(anonymize=True)


#### 2. Query the top 250 rated movies in the database

In [3]:
# Output of the top 250 movies
# Top 250 movies are output as list of json files
# Converting to json makes it easy to put into dataframe

top250 = json.dumps(imdb.top_250())


#### 3. Make a dataframe from the movie data

Keep the fields:

    num_votes
    rating
    tconst
    title
    year
    
And discard the rest

In [4]:
# Can use read_json to import the dump from avove
movies250 = (pd.read_json(top250))

# Subset only desired columns
movies = movies250[['num_votes', 'rating', 'tconst', 'title', 'year']]

#### 3. Select only the top 100 movies

In [5]:
movies = movies.ix[0:100,:]

In [ ]:
movies.tail()

#### 4. Get the genres and runtime for each movie and add them to the dataframe


There can be multiple genres per movie, so this will need some finessing.

In [7]:
# Create Empty lists to store genres/runtime
# movies_genres = []
# movies_runtime = []    


# for tconst in movies['tconst']: # iterate through the movie ids
#     title = imdb.get_title_by_id(tconst) # Get title info of id
#     movies_genres.append(title.genres)   # Append genre of id to list
#     movies_runtime.append(title.runtime) # Append 

In [8]:
# movies_genres

[[u'Crime', u'Drama'],
 [u'Crime', u'Drama'],
 [u'Crime', u'Drama'],
 [u'Action', u'Crime', u'Thriller'],
 [u'Biography', u'Drama', u'History'],
 [u'Crime', u'Drama'],
 [u'Crime', u'Drama'],
 [u'Adventure', u'Drama', u'Fantasy'],
 [u'Western'],
 [u'Drama'],
 [u'Adventure', u'Drama', u'Fantasy'],
 [u'Action', u'Adventure', u'Fantasy', u'Sci-Fi'],
 [u'Drama', u'Romance'],
 [u'Action', u'Mystery', u'Sci-Fi', u'Thriller'],
 [u'Action', u'Adventure', u'Drama', u'Fantasy'],
 [u'Drama'],
 [u'Biography', u'Crime', u'Drama'],
 [u'Action', u'Sci-Fi'],
 [u'Action', u'Adventure', u'Drama'],
 [u'Action', u'Adventure', u'Fantasy', u'Sci-Fi'],
 [u'Crime', u'Drama'],
 [u'Crime', u'Drama', u'Mystery', u'Thriller'],
 [u'Crime', u'Drama', u'Thriller'],
 [u'Drama', u'Family', u'Fantasy', u'Romance'],
 [u'Crime', u'Drama', u'Mystery', u'Thriller'],
 [u'Comedy', u'Drama', u'Romance', u'War'],
 [u'Crime', u'Drama', u'Thriller'],
 [u'Western'],
 [u'Animation', u'Adventure', u'Family', u'Fantasy'],
 [u'Action'

In [10]:
movies['genres'] = movies_genres
movies['runtime'] = movies_runtime

#### 4. Write the Results to a csv

In [13]:
# movies.to_csv('movies.csv',     # Filepath
#               encoding='utf-8') # use utf-8 encoding

---

## Part 2: Wrangle the text data

#### 1. Scrape the reviews for the top 100 movies

*Hint*: Use a loop to scrape each page at once

In [42]:
import requests
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

url = 'http://www.imdb.com/title/%s/' % 'tt0111161' # Set URL
response = requests.get(url)                        # Get URL Response
HTML = response.text                                # Obtain text from response

u'\n\n\n\n\n\n\n<!DOCTYPE html>\n<html\nxmlns:og="http://ogp.me/ns#"\nxmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///title/tt0111161?src=mdot">\n        \n    \n            <script type="text/javascript">var ue_t0=window.ue_t0||+new Date();</script>\n            <script type="text/javascript">\n                var ue_mid = "A1EVA'

In [43]:
# Select Text of Review
Selector(text=HTML).xpath('//span/div[2]/p/text()').extract() 

[u" One of the finest films made in recent years. It's a poignant story about hope. Hope gets me. That's what makes a film like this more than a movie. It tells a lesson about life. Those are the films people talk about 50 or even 100 years from you. It's also a story for freedom. Freedom from isolation,  from rule, from bigotry and hate. Freeman and Robbins are majestic in their performances. Each learns from the other. Their relationship is strong and you feel that from the first moment they make contact with one another. There is also a wonderful performance from legend James Whitmore as Brooks.",
 u'He shines when it is his time to go back into the world, only to find that the world grew up so fast he never even got a chance to blink. Stephen King\'s story is brought to the screen with great elegance and excitement. It is an extraordinary motion that people "will" be talking about in  50 or 100 years.  ']

In [45]:
# Select Review Score
Selector(text=HTML).xpath("//div[@class='ratingValue']/strong/span/text()").extract()

[u'9.3']

#### 2. Extract the reviews and the rating per review for each movie

*Note*: "soup" from BeautifulSoup is the html returned from all 25 pages. You'll need to either address each page individually or break them down by elements.

In [ ]:
# Reviews for all of the movies are stored as .gif files
# The file name corresponds to a score out of 100

#### 3. Remove the non AlphaNumeric characters from reviews

#### 4. Calculate the top 200 ngrams from the user reviews

Use the `TfidfVectorizer` in sklearn.

Recommended parameters:

    ngram_range = (1, 2)
    stop_words = 'english'
    binary = False
    max_features = 200

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### 5. Merge the user reviews and ratings

#### 6. Save this merged dataframe as a csv

---

## Part 3: Combine Tables in PostgreSQL

#### 1. Import your two .csv data files into your Postgre Database as two different tables

For ease, we can call these table1 and table2

#### 2. Connect to database and query the joined set

#### 3. Join the two tables 

#### 4. Select the newly joined table and save two copies of the into dataframes

---

## Part 4: Parsing and Exploratory Data Analysis

#### 1. Rename any columns you think should be renamed for clarity

#### 2. Describe anything interesting or suspicious about your data (quality assurance)

#### 3. Make four visualizations of interest to you using the data

---

## Part 5: Decision Tree Classifiers and Regressors

#### 1. What is our target attribute? 

Choose a target variable for the decision tree regressor and the classifier. 

#### 2. Prepare the X and Y matrices and preprocess data as you see fit

#### 3. Build and cross-validate your decision tree classifier

#### 4. Gridsearch optimal parameters for your classifier. Does the performance improve?

#### 5. Build and cross-validate your decision tree regressor

#### 6. Gridsearch the optimal parameters for your classifier. Does performance improve?

---

## Part 6: Elastic Net


#### 1. Gridsearch optimal parameters for an ElasticNet using the regression target and predictors you used for the decision tree regressor.


#### 2. Is cross-validated performance better or worse than with the decision trees? 

#### 3. Explain why the elastic net may have performed best at that particular l1_ratio and alpha

---

## Part 7: Bagging and Boosting: Random Forests, Extra Trees, and AdaBoost

#### 1. Load the random forest regressor, extra trees regressor, and adaboost regressor from sklearn

#### 2. Gridsearch optimal parameters for the three different ensemble methods.

#### 3. Evaluate the performance of the two bagging and one boosting model. Which performs best?

#### 4. Extract the feature importances from the Random Forest regressor and make a DataFrame pairing variable names with their variable importances.

#### 5. Plot the ranked feature importances.

#### 6.1 [BONUS] Gridsearch an optimal Lasso model and use it for variable selection (make a new predictor matrix with only the variables not zeroed out by the Lasso). 

#### 6.2 [BONUS] Gridsearch your best performing bagging/boosting model from above with the features retained after the Lasso. Does the score improve?

#### 7.1. [BONUS] Select a threshold for variable importance from your Random Forest regressor and use that to perform feature selection, creating a new subset predictor matrix.

#### 7.2 [BONUS] Using BaggingRegressor with a base estimator of your choice, test a model using the feature-selected dataset you made in 7.1

---

## [VERY BONUS] Part 8: PCA

#### 1. Perform a PCA on your predictor matrix

#### 2. Examine the variance explained and determine what components you want to keep based on them.

#### 3. Plot the cumulative variance explained by the ordered principal components.

#### 4. Gridsearch an elastic net using the principal components you selected as your predictors. Does this perform better than the elastic net you fit earlier?

#### 5. Gridsearch a bagging ensemble estimator that you fit before, this time using the principal components as predictors. Does this perform better or worse than the original? 

#### 6. Look at the loadings of the original predictor columns on the first 3 principal components. Is there any kind of intuitive meaning here?

Hint, you will probably want to sort by absolute value of magnitude of loading, and also only look at the obviously important (larger) ones!

# [Extremely Bonus] Part 9:  Clustering

![](https://snag.gy/jPSZ6U.jpg)

 ***Bonus Bonus:***
This extended bonus question is asking to do something we never really talked about but would like for you to attempt based on the assumptions that we learned during this weeks clustering lesson(s).

#### 1. Import your favorite clustering module

#### 2. Encode categoricals

#### 3. Evaluate cluster metics solely based on a range of K
If K-Means:  SSE/Inertia vs Silhouette (ie: Elbow), silhouette average, etc

#### 4.  Look at your data based on the subset of your predicted clusters.
Assign the cluster predictions back to your dataframe in order to see them in context.  This is great to be able to group by cluster to get a sense of the data that clumped together.

#### 5. Describe your findings based on the predicted clusters 
_How well did it do?  What's good or bad?  How would you improve this? Does any of it make sense?_